In [26]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import kagglehub
import os 

nba_stats_path = kagglehub.dataset_download("rodneycarroll78/nba-stats-1980-2024")
salary_path = kagglehub.dataset_download("ratin21/nba-player-stats-and-salaries-2000-2025")



100%|██████████| 536k/536k [00:00<00:00, 6.16MB/s]

Extracting files...


In [30]:
advanced_80to24 = pd.read_csv(os.path.join(nba_stats_path, 'Advanced.csv'))
ppg_80to24 = pd.read_csv(os.path.join(nba_stats_path, 'Player Per Game.csv'))
season_info_80to24 = pd.read_csv(os.path.join(nba_stats_path, 'Player Season Info.csv'))
salaries_00to25 = pd.read_csv(os.path.join(salary_path, 'NBA Player Stats and Salaries_2000-2025.csv'))
recent_season = pd.read_csv("C:/Users/Mik/OneDrive/Documents/2425regseason.csv")
